In [1]:
# Mount Google Drive và chuyển thư mục làm việc (tuỳ bạn chỉnh lại đường dẫn)
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/CS431

Mounted at /content/drive
/content/drive/MyDrive/CS431


In [2]:
import sys
sys.path.append("/content/drive/MyDrive/CS431")

# UI

In [3]:
import torch
from diffusers import StableDiffusionPipeline, DDIMScheduler, AutoencoderKL
from PIL import Image
import gradio as gr

# Import both IPAdapter classes with aliases
from IPAdapter.ip_adapter import IPAdapter, IPAdapterMLP, IPAdapterLoRA, IPAdapterLoRAMLP
from UniControlNet.src.train.UniControlNet import GlobalAdapterPipeline

# =========================
# 1. Khai báo đường dẫn model
# =========================

base_model_path = "SG161222/Realistic_Vision_V4.0_noVAE"
#base_model_path = "stablediffusionapi/rev-animated"
vae_model_path = "stabilityai/sd-vae-ft-mse"
image_encoder_path = "openai/clip-vit-base-patch32"

# Nhiều checkpoint IP-Adapter để lựa chọn
# Bây giờ IP_CKPT_OPTIONS sẽ chứa cả đường dẫn và lớp IPAdapter tương ứng
IP_CKPT_OPTIONS = {
    "IP-Adapter": {
        "path": "/content/drive/MyDrive/CS431/IPAdapter/models/ip-adapter_step250000.bin",
        "class": IPAdapter
    },
    "IP-Adapter MLP": {
        "path": "/content/drive/MyDrive/CS431/IPAdapter/modelMLP/ip-adapter_step250000.bin",
        "class": IPAdapterMLP
    },
    "IP-Adapter LoRA": {
        "image_proj_path": "/content/drive/MyDrive/CS431/IPAdapter/modelLoRA/image_proj_step250000.bin",
        "ip_lora_path": "/content/drive/MyDrive/CS431/IPAdapter/modelLoRA/ip_lora_step250000.safetensors",
        "class": IPAdapterLoRA
    },
    "IP-Adapter LoRA MLP": {
        "image_proj_path": "/content/drive/MyDrive/CS431/IPAdapter/modelLoRAMLP/image_proj_step250000.bin",
        "ip_lora_path": "/content/drive/MyDrive/CS431/IPAdapter/modelLoRAMLP/ip_lora_step250000.safetensors",
        "class": IPAdapterLoRAMLP
    },
    "UniControlNet": {
        "adapter_ckpt" : "/content/drive/MyDrive/CS431/UniControlNet/global_adapter_ckpt_finish/global_adapter_step_250000.pt",
        "class": GlobalAdapterPipeline
    },
}

device = "cuda"
print("Using device:", device)

# =========================
# 1.a. Tạo pipeline gốc 1 lần
# =========================

def build_pipe():
    print("Loading scheduler, VAE, and Stable Diffusion pipeline...")

    noise_scheduler = DDIMScheduler(
        num_train_timesteps=1000,
        beta_start=0.00085,
        beta_end=0.012,
        beta_schedule="scaled_linear",
        clip_sample=False,
        set_alpha_to_one=False,
        steps_offset=1,
    )

    vae = AutoencoderKL.from_pretrained(vae_model_path).to(dtype=torch.float16)

    pipe = StableDiffusionPipeline.from_pretrained(
        base_model_path,
        torch_dtype=torch.float16,
        scheduler=noise_scheduler,
        vae=vae,
        feature_extractor=None,
        safety_checker=None
    )

    print("Base pipeline loaded.")
    return pipe

print("Initializing Stable Diffusion base pipeline, please wait...")
pipe = build_pipe()

# Cache cho các IP-Adapter ứng với từng checkpoint
ip_model_cache = {}

# def get_ip_model(model_name: str):
#     """
#     Lấy IP-Adapter theo tên model (key trong IP_CKPT_OPTIONS).
#     Nếu chưa load thì load và cache lại.
#     """
#     if model_name not in IP_CKPT_OPTIONS:
#         raise ValueError(f"Model '{model_name}' không tồn tại trong IP_CKPT_OPTIONS")
#     model_config = IP_CKPT_OPTIONS[model_name]
#     ckpt_path = model_config["path"]
#     ip_adapter_class = model_config["class"] # Lấy tham chiếu đến lớp IPAdapter

#     if model_name not in ip_model_cache:
#         print(f"Loading IP-Adapter: {model_name} ({ckpt_path}) ...")
#         # Sử dụng lớp IPAdapter chính xác được chỉ định trong cấu hình
#         ip_model_cache[model_name] = ip_adapter_class(pipe, image_encoder_path, ckpt_path, device)
#         print(f"IP-Adapter '{model_name}' loaded.")

#     return ip_model_cache[model_name]
# Biến toàn cục để theo dõi model nào đang "chiếm giữ" UNet
import gc # Cần import thư viện này để dọn rác bộ nhớ

# Khai báo biến toàn cục
current_active_model_name = None
active_ip_adapter = None

def get_ip_model(model_name: str):
    # Phải khai báo global để thay đổi được biến pipe bên ngoài
    global current_active_model_name, active_ip_adapter, pipe

    # 1. Kiểm tra tên model hợp lệ
    if model_name not in IP_CKPT_OPTIONS:
        raise ValueError(f"Model '{model_name}' không tồn tại")

    # 2. Nếu model đang yêu cầu chính là model đang chạy -> Trả về luôn (Không reload)
    if model_name == current_active_model_name and active_ip_adapter is not None:
        return active_ip_adapter

    # ============================================================
    # BƯỚC 3: HARD RESET - XÓA PIPE CŨ, TẠO PIPE MỚI
    # ============================================================
    print(f"🔄 Switching model to: {model_name}")
    print("🧹 Cleaning up old pipeline & releasing VRAM...")

    # Xóa tham chiếu adapter cũ
    active_ip_adapter = None

    # Xóa pipeline cũ để giải phóng VRAM
    if 'pipe' in globals() and pipe is not None:
        del pipe

    # Dọn rác bộ nhớ Python và CUDA (Bắt buộc để không bị Out of Memory)
    gc.collect()
    torch.cuda.empty_cache()

    # Tạo lại Pipe mới tinh khôi (Sạch 100%)
    print("🛠️ Rebuilding Stable Diffusion Pipeline...")
    pipe = build_pipe()

    # ============================================================
    # BƯỚC 4: KHỞI TẠO IP-ADAPTER TRÊN PIPE MỚI
    # ============================================================
    model_config = IP_CKPT_OPTIONS[model_name]
    ip_adapter_class = model_config["class"]

    try:
        if "ip_lora_path" in model_config:
            # Model có LoRA
            new_adapter = ip_adapter_class(
                sd_pipe=pipe,
                image_proj_path=model_config["image_proj_path"],
                ip_lora_path=model_config["ip_lora_path"],
                image_encoder_path=image_encoder_path,
                device=device
            )
        elif "path" in model_config:
            # Model thường
            new_adapter = ip_adapter_class(
                sd_pipe=pipe,
                image_encoder_path=image_encoder_path,
                ip_ckpt=model_config["path"],
                device=device
            )
        elif "adapter_ckpt" in model_config:
            new_adapter = ip_adapter_class(
                sd15_name=base_model_path,
                adapter_ckpt=model_config["adapter_ckpt"],
                device=device
            )
        else:
            raise ValueError(f"Config lỗi cho {model_name}")

    except TypeError as e:
        print(f"❌ Lỗi khởi tạo: {e}")
        raise e

    # 5. Cập nhật trạng thái
    active_ip_adapter = new_adapter
    current_active_model_name = model_name

    print(f"✅ IP-Adapter '{model_name}' activated.")
    return active_ip_adapter

Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


Using device: cuda
Initializing Stable Diffusion base pipeline, please wait...
Loading scheduler, VAE, and Stable Diffusion pipeline...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

text_encoder/model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


Base pipeline loaded.


In [4]:
# =========================
# 2. Hàm sinh ảnh dùng IP-Adapter
# =========================

def generate_with_ip_adapter(
    model_name,
    image,
    prompt,
    num_samples,
    num_inference_steps,
    scale,
    seed
):
    # Lấy đúng IP-Adapter theo model mà user chọn
    ip_model = get_ip_model(model_name)

    if image is None:
        return []

    # Đảm bảo ảnh là PIL
    if not isinstance(image, Image.Image):
        image = Image.fromarray(image)

    # Resize nhẹ
    image = image.resize((512, 512))

    # Xử lý seed
    if seed is not None and seed != "":
        try:
            seed = int(seed)
        except ValueError:
            seed = 42
    else:
        seed = 42

    gen_kwargs = dict(
        pil_image=image,
        num_samples=int(num_samples),
        num_inference_steps=int(num_inference_steps),
        seed=seed,
        scale = float(scale)
    )

    # Nếu có text prompt → multimodal
    if prompt is not None and prompt.strip() != "":
        gen_kwargs["prompt"] = prompt

    images = ip_model.generate(**gen_kwargs)
    return images

In [5]:
CUSTOM_CSS = """
:root {
  color-scheme: dark;
}

body {
  margin: 0;
  padding: 0;
  background: linear-gradient(135deg, rgb(188 115 255), rgb(86 255 200)) !important;
}

/* Gradio wrapper */
.gradio-container,
.gradio-app,
#root {
  background: linear-gradient(135deg, rgb(188 115 255), rgb(86 255 200)) !important;
}

#ip-container {
  max-width: 1200px;
  margin: 0 auto;
  padding: 24px 12px 40px;
  font-family: system-ui, -apple-system, BlinkMacSystemFont, "Segoe UI", sans-serif;
  color: rgb(237 237 237);
}

/* Title box - Chuyển sang nền xám rất nhạt để nổi bật trên nền trắng */
#title-box {
  position: relative;
  padding: 18px 22px;
  border-radius: 12px;
  background: lightseagreen;
  color: #f9fafb;
  margin-bottom: 18px;
}

#title-box h1 {
  margin: 0;
  font-size: 23px;
  display: flex;
  align-items: center;
  gap: 10px;
  color: #f9fafb !important;
}

#title-box h1 .badge {
  border-radius: 6px;
  display: inline-flex;
  align-items: center;
  justify-content: center;
  width: 32px;
  height: 32px;
  background: black;
  color: #ffffff !important;
  font-size: 13px;
  font-weight: 600;
}

#title-box p {
  margin: 6px 0 0 0;
  font-size: 14px;
  color: #475569 !important;
}

#title-meta {
  margin-top: 10px;
  display: flex;
  gap: 6px;
}

#title-meta .chip {
  border-radius: 20px;
  font-weight: 500;
  font-size: 11px;
  padding: 4px 10px;
  background: #e2e8f0;
  color: #475569 !important;
  border: 1px solid #cbd5e1;
}

/* Main layout cards */
#left-panel,
#right-panel {
  border-radius: 12px;
  padding: 16px;
  border: 1px solid rgb(0 0 0);
  box-shadow: rgba(0, 0, 0, 0.05) 0px 4px 6px -1px;
  background: rgb(38 94 127) !important
}

/* Labels */
#ip-container label {
  font-weight: 600;
  font-size: 13px;
  color: #374151 !important;
  margin-bottom: 4px;
}

/* Inputs / Textbox / Dropdown - Ép nền trắng */
#ip-container .gradio-textbox,
#ip-container .gradio-dropdown,
#ip-container .gradio-slider,
#ip-container .gradio-image,
#ip-container .gradio-accordion {
  border-radius: 8px !important;
  background: #ffffff !important;
  border: 1px solid #d1d5db !important;
}

/* Nội dung bên trong input */
#ip-container textarea,
#ip-container input[type="text"],
#ip-container input[type="number"] {
  background-color: #ffffff !important;
  color: #000000 !important;
}

#ip-container select {
    background-color: #ffffff !important;
    color: #000000 !important;
}

/* Input placeholder */
#ip-container textarea::placeholder,
#ip-container input::placeholder {
  color: #9ca3af !important;
}

/* Buttons */
#ip-container button {
  font-weight: 600;
  border-radius: 8px;
}

/* Nút Generate - Màu xanh */
#ip-container button.primary {
  background: #2563eb !important;
  color: #ffffff !important;
  border: none !important;
}

#ip-container button.primary:hover {
  background: #1d4ed8 !important;
  box-shadow: 0 4px 12px rgba(37, 99, 235, 0.3);
}

/* Nút Clear - Màu trắng viền xám */
#ip-container button.secondary {
  background: rgb(226 224 224) !important;
  color: black !important;
  border: 1px solid rgb(226 224 224) !important;
}

/* Gallery */
#gallery {
  border-radius: 12px;
  padding: 8px;
  background: #f9fafb !important; /* Xám rất nhạt để phân biệt với nền */
  border: 1px dashed #d1d5db;
}

/* Tips Box */
#tips-box {
  font-size: 13px;
  color: #1e40af !important; /* Chữ xanh đậm */
  margin-top: 12px;
  padding: 12px;
  border-radius: 8px;
  background: black; /* Nền xanh rất nhạt */
  border: 1px solid black;
}

#tips-box ul {
  margin: 6px 0 0 16px;
  padding: 0;
}

/* Accordion Header */
#ip-container .gradio-accordion > button {
    background: #f3f4f6 !important;
    color: #111827 !important;
}

/* Image Upload Area */
#ip-container .gradio-image button {
    color: #374151 !important;
}
"""

def build_ui():
    with gr.Blocks(css=CUSTOM_CSS) as demo:
        with gr.Column(elem_id="ip-container"):
            gr.HTML("""
            <div id="title-box" style="display: flex; flex-direction: column; align-items: center;">
              <h1>
                <span class="badge">IP</span>
                <span>MÔ HÌNH TẠO SINH ẢNH</span>
              </h1>

              <p>
                Upload ảnh làm image prompt, thêm text prompt nếu muốn, sau đó bấm
                <b>Generate</b> để sinh ảnh mới.
              </p>
            </div>

            """)

            with gr.Row(elem_id="main-row"):
                # Cột trái
                with gr.Column(scale=1, min_width=320, elem_id="left-panel"):
                    # Dropdown chọn checkpoint
                    model_dropdown = gr.Dropdown(
                        label="Chọn IP-Adapter checkpoint",
                        choices=list(IP_CKPT_OPTIONS.keys()),
                        value=list(IP_CKPT_OPTIONS.keys())[0],
                    )

                    image_input = gr.Image(
                        label="Image Prompt 🎨",
                        type="pil"
                    )

                    prompt_input = gr.Textbox(
                        label="Text Prompt (optional) 📝",
                        placeholder="Ví dụ: a band play music in a garden",
                        lines=2
                    )



                    with gr.Accordion("Advanced Settings", open=False):
                        num_samples_input = gr.Slider(
                            minimum=1,
                            maximum=6,
                            value=4,
                            step=1,
                            label="Number of samples"
                        )

                        steps_input = gr.Slider(
                            minimum=10,
                            maximum=75,
                            value=50,
                            step=1,
                            label="Inference steps"
                        )
                        scale_input = gr.Slider(
                            minimum=0.0,
                            maximum=1.0,
                            value=0.6,
                            step=0.05,
                            label="IP-Adapter scale"
                        )

                        seed_input = gr.Textbox(
                            label="Seed (để trống = 42)",
                            value=""
                        )

                    with gr.Row():
                        generate_btn = gr.Button("🚀 Generate", variant="primary")
                        clear_btn = gr.Button("🧹 Clear output")

                # Cột phải
                with gr.Column(scale=2, elem_id="right-panel"):
                    gallery_output = gr.Gallery(
                        label="Generated Images",
                        show_label=True,
                        elem_id="gallery",
                        columns=2,
                        height=480
                    )

                    gr.HTML("""
                    <div id="tips-box">
                      <b>Tips: Nếu ảnh tạo ra không tốt hãy điều chỉnh scale</b>
                      <ul>
                        <li>Scale thấp : bám text nhiều.</li>
                        <li>Scale cao : bám ảnh gốc nhiều.</li>
                      </ul>
                    </div>
                    """)

        # Nối với hàm generate_with_ip_adapter
        generate_btn.click(
            fn=generate_with_ip_adapter,
            inputs=[
                model_dropdown,      # model được chọn
                image_input,
                prompt_input,
                num_samples_input,
                steps_input,
                scale_input,
                seed_input,
            ],
            outputs=gallery_output
        )

        clear_btn.click(fn=lambda: [], inputs=None, outputs=gallery_output)

    return demo

In [6]:
# Launch UI
ui = build_ui()
ui.launch(share=True)  # hoặc ui.launch() nếu chỉ muốn localhost

/tmp/ipython-input-354246180.py:191: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=CUSTOM_CSS) as demo:


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c0b90acb8237817ed6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
